In [2]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord, CartesianRepresentation, SphericalRepresentation, matrix_utilities

from matplotlib import cm, colors, pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# Imports from Data Lab notebook
from getpass import getpass

from dl.helpers.utils import convert
from dl import authClient as ac, queryClient as qc

%matplotlib inline

In [3]:
def input_query(object_name, radius_deg):
    # SkyCoord.from_name outputs the coordinates of a particular object given its name as a string
    c = SkyCoord.from_name(object_name)
    
    # Construct a Cone Search query
    # bp_rp<2
    query = """SELECT ra,dec,parallax,pmra,pmdec,nest4096,phot_g_mean_mag,phot_bp_mean_mag FROM gaia_dr2.gaia_source 
            WHERE (q3c_radial_query(ra, dec, {0}, {1}, %s)
            AND NOT parallax='Nan') AND astrometric_params_solved=31 AND parallax < 5. AND bp_rp<2""".format(c.ra.value,c.dec.value) % (radius_deg)
    
    # Submit the query
    response = qc.query(sql=query, fmt='csv')
    
    # Convert the query to a pandas DataFrame
    df = convert(response,'pandas')
    
    # Check the number of objects in your query
    print("The query of %s contains %s objects." % (object_name, len(df)))
    
    # Return the output DataFrame
    return df

In [4]:
# Pick your favorite object!

# Define an object
object_queried = "LMC"

# Define a radius
radius = 5

# Submit the function that returns a queried DataFrame
df = input_query(object_queried, radius)

The query of LMC contains 9116235 objects.


In [5]:
from astropy.table import Table
tab=Table.from_pandas(df)

In [6]:
tab.write('lmc_gaia.fits',format='fits')

In [8]:
from dl import storeClient as sc

sc.put(to='vos://public/lmc_gaia.fits',fr='./lmc_gaia.fits')

(1 / 1) ./lmc_gaia.fits -> vos://public/lmc_gaia.fits


'OK'

In [20]:
def define_variables (df):
    ra, dec = df["ra"], df["dec"]
    ra_rad, dec_rad = np.radians(ra), np.radians(dec)
    pmra, pmdec = df["pmra"], df["pmdec"]
    parallax = df['parallax']
    g, g_bp = df["phot_g_mean_mag"], df["phot_g_mean_mag"]-df["phot_bp_mean_mag"]
    return ra, dec, ra_rad, dec_rad, pmra, pmdec, parallax, g, g_bp

# Submit the function that defines variables from the DataFrame
ra, dec, ra_rad, dec_rad, pmra, pmdec, parallax, g, g_bp = define_variables(df)

In [11]:
def panel_plot(ra, dec, pmra, pmdec, color, mag):
    fig = plt.figure(figsize=(26,8))
    
    # A simple spatial plot
    ax1 = fig.add_subplot(131)
    ax1.scatter(ra, dec, marker='.', alpha= 1/np.power(len(df), 0.1))
    ax1.set_title("Spatial Plot",fontsize=10)
    ax1.set_xlabel("Right Ascension",fontsize=10)
    ax1.set_ylabel("Declination",fontsize=10)
    
    # A quiver plot (spatial plot with arrows that describe each object's proper motion)
    ax2 = fig.add_subplot(132)
    ax2.quiver(ra, dec, pmra, pmdec)
    ax2.set_title("Proper Motions",fontsize=10)
    ax2.set_xlabel("Right Ascension",fontsize=10)
    ax2.set_ylabel("Declination",fontsize=10)
    
    # A color-magnitude diagram that exhibits object density
    ax3 = fig.add_subplot(133)
    im3 = ax3.hexbin(g_bp, g, bins='log', cmap=plt.cm.viridis, mincnt=1)
    ax3.set_ylim(max(g), min(g))
    ax3.set_title("Color-Magnitude Diagram",fontsize=10)
    ax3.set_xlabel("Gaia G-BP Color",fontsize=10)
    ax3.set_ylabel("Gaia G Magnitude",fontsize=10)
    cb = plt.colorbar(im3,label='log(N)')

In [9]:
df

,ra,dec,parallax,pmra,pmdec,nest4096,phot_g_mean_mag,phot_bp_mean_mag
0,93.667119,-72.613311,0.180944,1.832755,0.250782,153259513.0,18.942707,19.404827
1,93.588058,-72.659901,0.477063,0.494848,0.290034,153259504.0,20.225683,20.324703
2,93.593370,-72.657630,0.168500,-4.158330,7.319939,153259504.0,14.472233,14.895664
3,93.591467,-72.656937,0.006647,1.984393,2.379452,153259504.0,19.940767,19.191557
4,93.587913,-72.655535,0.250662,4.433331,2.117632,153259504.0,19.361525,20.197117
5,93.595867,-72.654451,0.048317,2.101609,0.368403,153259504.0,19.421318,19.851545
6,93.608310,-72.641879,-0.034217,1.628020,0.941230,153259506.0,20.245707,20.366093
7,93.597815,-72.649703,-1.115696,0.840547,2.536073,153259504.0,20.060421,20.235312
8,93.593162,-72.651476,-0.275557,3.238420,0.345037,153259504.0,20.543503,20.895430
9,93.594225,-72.649424,0.059188,2.718333,1.712675,153259504.0,19.892540,20.367086


In [ ]:
def convert_to_skycoord (g, ra_rad, dec_rad, parallax, dist1, dist2):
    # Define variables as as an numpy array and SkyCoord object
    # Note: The SkyCoord object contains data for ra, dec, and parallax
    mag = np.array(g)
    scnow = SkyCoord(np.array(ra_rad) * u.rad, np.array(dec_rad) * u.rad, 
                 distance=(np.array(parallax)*u.mas).to(u.pc, u.parallax()))

    # Create a "mask" with varying data thresholds (here depending on magnitude and distance)
    msk = (mag < 25)  #& (scnow.distance > dist1*u.pc) & (scnow.distance < dist2*u.pc)
    
    # Apply data thresholds to the given magnitude and SkyCoord objects
    mag = mag[msk]
    scnow = scnow[msk]
    
    # Print the new object count after applying data thresholds
    print('After the thresholds are applied, this query contains %s objects.' % (len(scnow)))
    
    # Return the defined variables
    return mag, msk, scnow
    
#mag, msk, scnow = convert_to_skycoord(g, ra_rad, dec_rad, parallax, 1e2, 9e2)

In [10]:
panel_plot(df['ra'], df['dec'], df['pmra'], df['pmdec'], df['phot_g_mean_mag']-df['phot_bp_mean_mag'],df['phot_g_mean_mag'])

NameError: name 'panel_plot' is not defined